In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from common_variables import *
from time_series import *

/Users/tahaatiq/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
epochs = 50
np.random.seed(seed)
tf.random.set_seed(seed)

In [4]:
df = pd.read_csv(train_time_series_path, dayfirst=True, 
    index_col=['Date'], parse_dates=['Date'])
df

,HLAvg,MA,Returns
Date,,,
2010-01-01 00:39:00,1.616600,1.616659,-2.209139e-06
2010-01-01 00:40:00,1.616650,1.616659,-4.418283e-07
2010-01-01 00:41:00,1.616650,1.616661,1.767312e-06
2010-01-01 00:42:00,1.616675,1.616663,1.104569e-06
2010-01-01 00:43:00,1.616700,1.616666,1.767307e-06
...,...,...,...
2022-07-08 09:25:00,1.195500,1.194459,1.070477e-04
2022-07-08 09:26:00,1.195635,1.194613,1.288606e-04
2022-07-08 09:27:00,1.195410,1.194745,1.112074e-04


In [ ]:
fig = plt.figure(figsize=(24, 18))
ax1, ax2, ax3 = fig.subplots(3)
ax1.set_title('HLAvg')
ax1.set(xlabel='Date', ylabel='High-Low Average')
ax1.plot(df['HLAvg'])
ax2.set_title('MA')
ax2.set(xlabel='Date', ylabel='MA')
ax2.plot(df['MA'])
ax3.set_title('Returns')
ax3.set(xlabel='Date', ylabel='Returns')
ax3.plot(df['Returns'])

In [ ]:
scaler = MinMaxScaler()
train_values = scaler.fit_transform(df[['Returns']].values)

In [ ]:
fig = plt.figure(figsize=(24, 8))
ax1 = fig.subplots(1)
ax1.set_title('Returns MinMax Scaled')
ax1.set(xlabel='Sample', ylabel='Scaled Returns')
ax1.plot(train_values)

In [ ]:
X, y = get_train(train_values, window_size)

In [ ]:
df_val = pd.read_csv(validate_time_series_path, dayfirst = True, usecols=['Date','Returns'],
    index_col=['Date'], parse_dates=['Date'])
df_val['Scaled'] = scaler.transform(df_val[['Returns']].values)
X_val, y_val = get_val(df_val['Scaled'].values, window_size)

In [ ]:
model = Sequential()
model.add(LSTM(76, input_shape=(X.shape[1], 1), return_sequences = False))
model.add(Dropout(0.2))
model.add(Dense(1))
optimizer = tf.keras.optimizers.Adam()
model.compile(loss="mse", optimizer=optimizer)

print(model.summary())

In [ ]:
history = model.fit(X, y, validation_data=(X_val, y_val), epochs = epochs, batch_size = batch_size, shuffle=False, verbose = 2)

In [ ]:
save_model(model, model_path)
joblib.dump(scaler, scaler_path) 

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax1 = fig.subplots(1)
ax1.set_title('Model Loss')
ax1.set(xlabel='Epoch', ylabel='Loss')
ax1.plot(history.history['loss'][7:], label='Train Loss')
ax1.plot(history.history['val_loss'][7:], label='Val Loss')
ax1.legend()